# Ditto for Entity Resolution (FAIR-DA4ER)

This notebook implements the **Ditto** method for **Entity Resolution (ER)** as described in the [FAIR-DA4ER](https://github.com/MarcoNapoleone/FAIR-DA4ER) repository.

## Overview (from README)

- **FAIR-DA4ER** provides code for training **Ditto** models for Entity Resolution, with optional **FAIR-DA4ER** data augmentation.
- **Ditto** (Li et al.) casts Entity Matching as a **sequence-pair classification** problem using pre-trained LMs (e.g. BERT, DistilBERT).
- **Data format**: Each example is a line: `record1 \t record2 \t label`, where each record is serialized as `COL attr_name VAL attr_value COL ...` and label is `0` (no match) or `1` (match).
- **Task config**: Datasets are defined in `configs.json` with `trainset`, `validset`, `testset` paths.

## 1. Clone FAIR-DA4ER repo and install dependencies

All configs, data paths, and code come from the [FAIR-DA4ER](https://github.com/MarcoNapoleone/FAIR-DA4ER) repository. Run the cell below to clone the repo and install from its `requirements.txt`. Subsequent cells run from the repo root.

In [8]:
# Clone FAIR-DA4ER repo (skip if already cloned)
import os
REPO_DIR = "FAIR-DA4ER"
if not os.path.isdir(REPO_DIR):
    !git clone https://github.com/MarcoNapoleone/FAIR-DA4ER.git {REPO_DIR}
%cd {REPO_DIR}

# Install dependencies from repo's requirements.txt
!pip install -q -r requirements.txt

Cloning into 'FAIR-DA4ER'...


remote: Enumerating objects: 469, done.
remote: Counting objects: 100% (469/469), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 469 (delta 232), reused 467 (delta 230), pack-reused 0 (from 0)
Receiving objects: 100% (469/469), 35.38 MiB | 14.50 MiB/s, done.
Resolving deltas: 100% (232/232), done.
Updating files: 100% (160/160), done.
/workspace/Ditto/FAIR-DA4ER/FAIR-DA4ER


In [11]:
# Install dependencies into the same Python this kernel uses
import sys
!{sys.executable} -m pip install -q numpy torch transformers scikit-learn


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python3.13 -m pip install --upgrade pip


## 2. Imports

In [12]:
import os
import json
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/usr/local/lib/python3.13/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [14]:
cd ditto

/workspace/Ditto/FAIR-DA4ER/FAIR-DA4ER/ditto


## 3. Load task configuration (configs.json from repo)

In [25]:
# Load config: use IJF_sentence-bert (data/ijf/sentence-bert/). Paths relative to ditto/.
CONFIG_PATH = "configs.json"
if not os.path.isfile(CONFIG_PATH):
    CONFIG_PATH = os.path.join(os.path.dirname(os.path.abspath(".")), "configs.json")
with open(CONFIG_PATH) as f:
    configs = json.load(f)
configs_by_name = {c["name"]: c for c in configs}

TASK_NAME = "IJF_sentence-bert"
DITTO_DIR = os.path.dirname(os.path.abspath(CONFIG_PATH))

# Resolve DITTO_DIR: if data/ijf/sentence-bert/train.txt is not here, walk up until we find it (fixes nested cd)
IJF_TRAIN_REL = os.path.join("data", "ijf", "sentence-bert", "train.txt")
while DITTO_DIR != os.path.dirname(DITTO_DIR):
    if os.path.isfile(os.path.join(DITTO_DIR, IJF_TRAIN_REL)):
        break
    DITTO_DIR = os.path.dirname(DITTO_DIR)

if TASK_NAME in configs_by_name:
    config = configs_by_name[TASK_NAME]
    trainset_path = os.path.join(DITTO_DIR, config["trainset"])
    validset_path = os.path.join(DITTO_DIR, config["validset"])
    testset_path = os.path.join(DITTO_DIR, config["testset"])
else:
    trainset_path = os.path.join(DITTO_DIR, "data", "ijf", "sentence-bert", "train.txt")
    validset_path = os.path.join(DITTO_DIR, "data", "ijf", "sentence-bert", "valid.txt")
    testset_path = os.path.join(DITTO_DIR, "data", "ijf", "sentence-bert", "test.txt")
    print(f"(IJF_sentence-bert not in configs.json; using paths under {DITTO_DIR})")

print(f"Task: {TASK_NAME}")
print(f"DITTO_DIR: {DITTO_DIR}")
print(f"Train: {trainset_path}, Valid: {validset_path}, Test: {testset_path}")
print(f"Train exists: {os.path.isfile(trainset_path)}, Valid: {os.path.isfile(validset_path)}, Test: {os.path.isfile(testset_path)}")

(IJF_sentence-bert not in configs.json; using paths under /workspace/Ditto/FAIR-DA4ER/FAIR-DA4ER/ditto)
Task: IJF_sentence-bert
DITTO_DIR: /workspace/Ditto/FAIR-DA4ER/FAIR-DA4ER/ditto
Train: /workspace/Ditto/FAIR-DA4ER/FAIR-DA4ER/ditto/data/ijf/sentence-bert/train.txt, Valid: /workspace/Ditto/FAIR-DA4ER/FAIR-DA4ER/ditto/data/ijf/sentence-bert/valid.txt, Test: /workspace/Ditto/FAIR-DA4ER/FAIR-DA4ER/ditto/data/ijf/sentence-bert/test.txt
Train exists: True, Valid: True, Test: True


## 4. Ditto dataset (Ditto serialization format)

Each line: `record1 \t record2 \t label`. Records use `COL` / `VAL` tokens.

In [28]:
ls

3-gram/                  build_train_valid_test.py           sentence-bert/
blocking_sentence_bert/  pro_supplier_standardization_v.csv


In [29]:
class DittoDataset(Dataset):
    """Dataset for Ditto ER: pairs of serialized records + binary label."""

    def __init__(self, path, tokenizer, max_len=256, size=None):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pairs = []
        self.labels = []

        lines = open(path).readlines() if isinstance(path, str) else path
        for line in lines:
            line = line.strip()
            if not line:
                continue
            parts = line.split("\t")
            if len(parts) != 3:
                continue
            s1, s2, label = parts[0], parts[1], int(parts[2])
            self.pairs.append((s1, s2))
            self.labels.append(label)

        if size:
            self.pairs = self.pairs[:size]
            self.labels = self.labels[:size]

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        left, right = self.pairs[idx]
        # Compatible with old (encode_plus) and new (callable) transformers
        tokenizer_fn = getattr(self.tokenizer, "encode_plus", self.tokenizer)
        enc = tokenizer_fn(
            left,
            right,
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

    @staticmethod
    def collate_fn(batch):
        return {
            "input_ids": torch.stack([b["input_ids"] for b in batch]),
            "attention_mask": torch.stack([b["attention_mask"] for b in batch]),
            "labels": torch.stack([b["labels"] for b in batch]),
        }

# Model name (same as FAIR-DA4ER / megagonlabs ditto)
LM_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(LM_NAME)

MAX_LEN = 256

# CPU-friendly: randomly subsample train/valid/test to fixed sizes
N_TRAIN_IJF = 2000
N_VALID_IJF = 250
N_TEST_IJF = 250
SEED_IJF = 42

random.seed(SEED_IJF)
np.random.seed(SEED_IJF)

def load_and_sample(path, n_target):
    """Load file and randomly sample n_target lines (or all if file has fewer)."""
    with open(path) as f:
        lines = [line.strip() for line in f if line.strip()]
    if len(lines) <= n_target:
        return lines
    idx = np.random.choice(len(lines), size=n_target, replace=False)
    return [lines[i] for i in sorted(idx)]

train_lines = load_and_sample(trainset_path, N_TRAIN_IJF)
valid_lines = load_and_sample(validset_path, N_VALID_IJF)
test_lines = load_and_sample(testset_path, N_TEST_IJF)

train_ds = DittoDataset(train_lines, tokenizer, max_len=MAX_LEN)
valid_ds = DittoDataset(valid_lines, tokenizer, max_len=MAX_LEN)
test_ds = DittoDataset(test_lines, tokenizer, max_len=MAX_LEN)

print(f"Train: {len(train_ds)}, Valid: {len(valid_ds)}, Test: {len(test_ds)} (random subsample)")
print(f"Match rate (train): {100 * np.mean(train_ds.labels):.1f}%")
print("Sample:", train_ds.pairs[0], "->", train_ds.labels[0])

Train: 1998, Valid: 250, Test: 249 (random subsample)
Match rate (train): 37.8%
Sample: ('COL name VAL WORLD FUEL SERVICES COL name_clean VAL WORLD FUEL SERVICES COL sources VAL ["fd"] COL count VAL 1520 COL lobby_count VAL 0 COL amount VAL 41556465.18 COL amount_bc VAL null COL example VAL WORLD FUEL SERVICES', 'COL name VAL SERVICE FUEL SERVICE CANADA COL name_clean VAL SERVICE FUEL SERVICE CANADA COL sources VAL ["fd"] COL count VAL 1 COL lobby_count VAL 0 COL amount VAL 0 COL amount_bc VAL null COL example VAL service fuel service canada') -> 1


## 5. Ditto model (LM + linear head for binary classification)

In [30]:
class DittoModel(nn.Module):
    """Ditto: pre-trained LM with a linear layer for binary match classification."""

    def __init__(self, lm_name="distilbert-base-uncased", num_labels=2):
        super().__init__()
        self.bert = AutoModel.from_pretrained(lm_name)
        hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # [CLS] representation
        pooled = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(pooled)
        return logits

## 6. Training loop

In [31]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

BATCH_SIZE = 8
LR = 3e-5
N_EPOCHS = 5

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=DittoDataset.collate_fn,
)
valid_loader = DataLoader(
    valid_ds,
    batch_size=BATCH_SIZE * 2,
    shuffle=False,
    collate_fn=DittoDataset.collate_fn,
)
test_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE * 2,
    shuffle=False,
    collate_fn=DittoDataset.collate_fn,
)

model = DittoModel(lm_name=LM_NAME).to(device)
optimizer = AdamW(model.parameters(), lr=LR)
num_training_steps = len(train_loader) * N_EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
criterion = nn.CrossEntropyLoss()

Loading weights: 100%|██████████| 100/100 [00:00<00:00, 702.36it/s, Materializing param=transformer.layer.5.sa_layer_norm.weight]   
DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [32]:
def evaluate(model, loader, threshold=None):
    """Evaluate model. If threshold is None, find optimal F1 threshold; else use it."""
    model.eval()
    all_labels, all_probs = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"]
            logits = model(input_ids, attention_mask)
            probs = logits.softmax(dim=1)[:, 1].cpu().numpy()
            all_labels.extend(labels.numpy())
            all_probs.extend(probs)
    if threshold is not None:
        preds = [1 if p > threshold else 0 for p in all_probs]
        f1 = f1_score(all_labels, preds)
        return f1, threshold, preds, all_labels
    best_f1, best_th = 0.0, 0.5
    for th in np.arange(0.0, 1.0, 0.05):
        p = [1 if x > th else 0 for x in all_probs]
        f1 = f1_score(all_labels, p)
        if f1 > best_f1:
            best_f1, best_th = f1, th
    pred_best = [1 if x > best_th else 0 for x in all_probs]
    return best_f1, best_th, pred_best, all_labels

def train_epoch(model, loader, optimizer, scheduler, criterion):
    model.train()
    total_loss = 0.0
    for batch in loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [70]:
best_dev_f1 = 0.0
best_test_f1 = 0.0

for epoch in range(1, N_EPOCHS + 1):
    loss = train_epoch(model, train_loader, optimizer, scheduler, criterion)
    dev_f1, dev_th, _, _ = evaluate(model, valid_loader)
    test_f1, _, _, _ = evaluate(model, test_loader, threshold=dev_th)
    if dev_f1 > best_dev_f1:
        best_dev_f1 = dev_f1
        best_test_f1 = test_f1

    print(f"Epoch {epoch} | loss={loss:.4f} | dev_f1={dev_f1:.4f} | test_f1={test_f1:.4f} | best_test_f1={best_test_f1:.4f}")

Epoch 1 | loss=0.5062 | dev_f1=0.9043 | test_f1=0.9297 | best_test_f1=0.9297
Epoch 2 | loss=0.1321 | dev_f1=0.9462 | test_f1=0.9670 | best_test_f1=0.9670
Epoch 3 | loss=0.0673 | dev_f1=0.9348 | test_f1=0.9724 | best_test_f1=0.9670
Epoch 4 | loss=0.0290 | dev_f1=0.9362 | test_f1=0.9462 | best_test_f1=0.9670
Epoch 5 | loss=0.0169 | dev_f1=0.9355 | test_f1=0.9565 | best_test_f1=0.9670


## 7. Final test report (using validation threshold)

In [71]:
dev_f1, dev_th, _, _ = evaluate(model, valid_loader)
test_f1, _, test_preds, test_labels = evaluate(model, test_loader, threshold=dev_th)

print("Final test set performance (threshold from validation):")
print(classification_report(test_labels, test_preds, target_names=["No match", "Match"]))
print(f"F1: {f1_score(test_labels, test_preds):.4f}, Precision: {precision_score(test_labels, test_preds):.4f}, Recall: {recall_score(test_labels, test_preds):.4f}")

Final test set performance (threshold from validation):
              precision    recall  f1-score   support

    No match       0.98      0.97      0.97       158
       Match       0.95      0.97      0.96        91

    accuracy                           0.97       249
   macro avg       0.96      0.97      0.97       249
weighted avg       0.97      0.97      0.97       249

F1: 0.9565, Precision: 0.9462, Recall: 0.9670


In [2]:
# Inspect FP, FN, TP, TN examples on the IJF sentence-bert subset
import numpy as np

# Convert labels/predictions to numpy arrays
y_true = np.array(test_labels)
y_pred = np.array(test_preds)

# Indices for each case
tp_idx = np.where((y_true == 1) & (y_pred == 1))[0]
fp_idx = np.where((y_true == 0) & (y_pred == 1))[0]
tn_idx = np.where((y_true == 0) & (y_pred == 0))[0]
fn_idx = np.where((y_true == 1) & (y_pred == 0))[0]

print(f"TP: {len(tp_idx)}, FP: {len(fp_idx)}, TN: {len(tn_idx)}, FN: {len(fn_idx)}")


def show_examples(ds, idxs, kind, k=5):
    """Pretty-print up to k examples for a given index set."""
    print(f"\n=== {kind} examples (showing up to {k}) ===")
    for idx in idxs[:k]:
        rec1, rec2 = ds.pairs[idx]
        label = ds.labels[idx]
        print(f"\nIndex: {idx}")
        print(f"True label: {label}")
        print(f"Record 1: {rec1}")
        print(f"Record 2: {rec2}")

# Show a few examples of each type from the test set
# show_examples(test_ds, tp_idx, "True Positives")
show_examples(test_ds, fp_idx, "False Positives")
# show_examples(test_ds, tn_idx, "True Negatives")
# show_examples(test_ds, fn_idx, "False Negatives")

NameError: name 'test_labels' is not defined

## 8. Predict on new pairs (optional)

Serialize two records in Ditto format (`COL attr VAL value ...`) and get match probability.

In [62]:
def predict_pair(model, tokenizer, record1, record2, max_len=256):
    """Predict match probability for a pair of Ditto-serialized records."""
    enc = tokenizer(
        record1,
        record2,
        max_length=max_len,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    model.eval()
    with torch.no_grad():
        logits = model(enc["input_ids"].to(device), enc["attention_mask"].to(device))
        prob_match = logits.softmax(dim=1)[0, 1].item()
    return prob_match

# Example
r1 = "COL title VAL samsung galaxy COL brand VAL samsung"
r2 = "COL title VAL samsung galaxy s21 COL brand VAL samsung"
prob = predict_pair(model, tokenizer, r1, r2)
print(f"Match probability: {prob:.4f} -> {'Match' if prob >= 0.5 else 'No match'}")

Match probability: 0.0973 -> No match


---
## Part B: Ditto with Data Augmentation (DA_iTunes-Amazon)

This section trains Ditto on **DA_iTunes-Amazon** using **in-training data augmentation (MixDA)** from the repo: the train set uses the `ditto_light` dataset with `da='del'` (span deletion), and the model mixes original and augmented representations during training. Valid/test use no augmentation. Results are reported as before.

In [16]:
# Ensure we're in the repo's ditto directory (paths are relative to it)
import os
try:
    _ditto_dir = os.path.join(REPO_DIR, "ditto")
except NameError:
    _ditto_dir = os.path.join("FAIR-DA4ER", "ditto")
if os.path.isdir(_ditto_dir) and not os.path.isfile("configs.json"):
    os.chdir(_ditto_dir)
# Load config for DA_iTunes-Amazon (configs[1])
with open("configs.json") as f:
    configs_da = json.load(f)
TASK_NAME_DA = configs_da[1]["name"]  # DA_iTunes-Amazon
config_da = {c["name"]: c for c in configs_da}[TASK_NAME_DA]
trainset_path_da = config_da["trainset"]
validset_path_da = config_da["validset"]
testset_path_da = config_da["testset"]
print(f"Task (with DA): {TASK_NAME_DA}")
print(f"Train: {trainset_path_da}, Valid: {validset_path_da}, Test: {testset_path_da}")

Task (with DA): DA_iTunes-Amazon
Train: data/fair/iTunes-Amazon/train.txt, Valid: data/fair/iTunes-Amazon/valid.txt, Test: data/fair/iTunes-Amazon/test.txt


In [17]:
# Use ditto_light dataset with in-training augmentation (da='del') for train; no DA for valid/test
import sys
_ditto_abs = os.path.abspath(_ditto_dir)
if _ditto_abs not in sys.path:
    sys.path.insert(0, _ditto_abs)
from ditto_light.dataset import DittoDataset as DittoDatasetDA
from ditto_light.ditto import DittoModel as DittoModelDA

MAX_LEN_DA = 256
DA_OP = "del"  # span deletion (or 'all' for RandAugment)
train_ds_da = DittoDatasetDA(trainset_path_da, lm="distilbert", max_len=MAX_LEN_DA, da=DA_OP)
valid_ds_da = DittoDatasetDA(validset_path_da, lm="distilbert", max_len=MAX_LEN_DA, da=None)
test_ds_da = DittoDatasetDA(testset_path_da, lm="distilbert", max_len=MAX_LEN_DA, da=None)

BATCH_SIZE_DA = 8
train_loader_da = DataLoader(train_ds_da, batch_size=BATCH_SIZE_DA, shuffle=True, collate_fn=DittoDatasetDA.pad)
valid_loader_da = DataLoader(valid_ds_da, batch_size=BATCH_SIZE_DA * 2, shuffle=False, collate_fn=DittoDatasetDA.pad)
test_loader_da = DataLoader(test_ds_da, batch_size=BATCH_SIZE_DA * 2, shuffle=False, collate_fn=DittoDatasetDA.pad)
print(f"Train: {len(train_ds_da)}, Valid: {len(valid_ds_da)}, Test: {len(test_ds_da)} (with DA op={DA_OP} on train)")

Train: 321, Valid: 109, Test: 109 (with DA op=del on train)


In [ ]:
# Model with MixDA (mixes original + augmented [CLS] during training)
set_seed(42)
model_da = DittoModelDA(device=device, lm="distilbert", alpha_aug=0.8).to(device)
optimizer_da = AdamW(model_da.parameters(), lr=LR)
num_steps_da = len(train_loader_da) * N_EPOCHS
scheduler_da = get_linear_schedule_with_warmup(optimizer_da, num_warmup_steps=0, num_training_steps=num_steps_da)
criterion_da = nn.CrossEntropyLoss()

In [19]:
def train_epoch_da(model, loader, optimizer, scheduler, criterion):
    """Train one epoch; loader returns (x1, x2, y) when DA is used, (x, y) otherwise."""
    model.train()
    total_loss = 0.0
    for batch in loader:
        optimizer.zero_grad()
        if len(batch) == 3:
            x1, x2, y = batch
            logits = model(x1, x2)
        else:
            x, y = batch
            logits = model(x)
        loss = criterion(logits, y.to(device))
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate_da(model, loader, threshold=None):
    """Evaluate; valid/test loaders return (x, y) only."""
    model.eval()
    all_labels, all_probs = [], []
    with torch.no_grad():
        for batch in loader:
            if len(batch) == 3:
                x, _, y = batch
            else:
                x, y = batch
            logits = model(x)
            probs = logits.softmax(dim=1)[:, 1].cpu().numpy()
            all_labels.extend(y.numpy())
            all_probs.extend(probs)
    if threshold is not None:
        preds = [1 if p > threshold else 0 for p in all_probs]
        return f1_score(all_labels, preds), threshold, preds, all_labels
    best_f1, best_th = 0.0, 0.5
    for th in np.arange(0.0, 1.0, 0.05):
        p = [1 if x > th else 0 for x in all_probs]
        f1 = f1_score(all_labels, p)
        if f1 > best_f1:
            best_f1, best_th = f1, th
    pred_best = [1 if x > best_th else 0 for x in all_probs]
    return best_f1, best_th, pred_best, all_labels

---
## IJF Blocking with Sentence-BERT (top-20 only)

Load `pro_supplier_with_clean_and_canonical_trimmed.csv`, apply sentence-BERT blocking (top-20 per record), then create train/test/valid splits and train Ditto.

In [33]:
# Step 1: Load CSV and serialize records to Ditto format
import csv
import os
import sys
import re
import numpy as np
from collections import defaultdict

# Path to the CSV
CSV_PATH_BLOCKING = "data/ijf/blocking_sentence_bert/pro_supplier_with_clean_and_canonical_trimmed.csv"

# Ensure we're in ditto directory - try multiple paths
if not os.path.isfile(CSV_PATH_BLOCKING):
    CSV_PATH_BLOCKING = os.path.join(DITTO_DIR, CSV_PATH_BLOCKING)
if not os.path.isfile(CSV_PATH_BLOCKING):
    # Try relative to current working directory
    CSV_PATH_BLOCKING = os.path.join("FAIR-DA4ER", "ditto", CSV_PATH_BLOCKING)

# Load CSV and serialize to Ditto format
records_blocking = []
canonical_ints = []
with open(CSV_PATH_BLOCKING, encoding="utf-8", newline="") as f:
    reader = csv.DictReader(f)
    columns = [c for c in reader.fieldnames if c != "canonical_int"]  # exclude canonical_int from record text
    for i, row in enumerate(reader):
        rec_str = " ".join([f"COL {col} VAL {row.get(col, '').strip()}" for col in columns])
        records_blocking.append((str(i), rec_str))
        canonical_ints.append(int(row.get("canonical_int", 0)))

print(f"Loaded {len(records_blocking)} records from CSV")
print(f"Sample record: {records_blocking[0][1][:200]}...")
print(f"Canonical ints range: {min(canonical_ints)} to {max(canonical_ints)}")

Loaded 1388738 records from CSV
Sample record: COL clean_supplier_name VAL SIMZER DESIGN COL address VAL  COL city VAL  COL prov VAL  COL postal VAL  COL country VAL CA...
Canonical ints range: 1 to 157859


In [56]:
# Step 2: Random row sampling + sentence-BERT blocking (top-500 per sampled row)
# Strategy: Randomly sample rows WITHOUT replacement, find top-500 similar pairs per row
# Target: ~1.8M pairs total for <30 min training on RTX 5090, with 15-20% match ratio
# Increasing TOP_K to 500 helps reduce match percentage (more diverse pairs)
import numpy as np
try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sentence-transformers"])
    from sentence_transformers import SentenceTransformer
from tqdm import tqdm

BERT_BATCH_SIZE = 512
TOP_K = 1000  # top-500 most similar pairs per sampled row (increased to reduce match %)
# Target total pairs: ~1.8M for <30 min training on RTX 5090 GPU
# With top-500 per row: sample ~3,600 rows (fewer rows needed since more pairs per row)
TARGET_TOTAL_PAIRS = 100_000  # Adjust this to control training time
N_SAMPLE_ROWS = TARGET_TOTAL_PAIRS // TOP_K  # ~3,600 rows

_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device for encoding and top-k: {_device}")
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=_device)
lines = [rec for _, rec in records_blocking]
print(f"Total records: {len(lines):,}")

# Encode ALL records (needed for similarity search)
print("Encoding all records with sentence-BERT (batched)...")
vecs = model.encode(
    lines,
    batch_size=BERT_BATCH_SIZE,
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=True,
)
V_np = np.float32(vecs)
n_total = len(V_np)
del vecs

# Free GPU memory: drop the encoder model
if _device == "cuda":
    del model
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print(f"Freed encoder model; GPU memory available for blocking.")

# Move embeddings to GPU for fast matmul + topk
use_gpu = _device == "cuda"
if use_gpu:
    V = torch.from_numpy(V_np).to(_device)
else:
    V = V_np

# Randomly sample rows to process (WITHOUT replacement - each row selected only once)
np.random.seed(42)
n_sample = min(N_SAMPLE_ROWS, n_total)
sampled_indices = np.random.choice(n_total, size=n_sample, replace=False)  # replace=False ensures no duplicates
sampled_indices = np.sort(sampled_indices)  # Sort for better cache locality
print(f"\nSampling {n_sample:,} rows (out of {n_total:,}) WITHOUT replacement")
print(f"Each sampled row will generate top-{TOP_K} similar pairs")
print(f"Expected total: ~{n_sample * TOP_K:,} pairs (~{TARGET_TOTAL_PAIRS:,} target)")
print(f"Target: <30 min training time on RTX 5090")

# Process sampled rows: for each sampled row, find top-500 similar pairs
pairs_bert = []
BATCH_SIZE_SAMPLE = 2000  # Process sampled rows in batches to manage GPU memory

for batch_start in tqdm(range(0, len(sampled_indices), BATCH_SIZE_SAMPLE), desc=f"Top-{TOP_K} blocking"):
    batch_end = min(batch_start + BATCH_SIZE_SAMPLE, len(sampled_indices))
    batch_indices = sampled_indices[batch_start:batch_end]
    batch_size = len(batch_indices)
    
    if use_gpu:
        # Get embeddings for this batch of sampled rows
        batch_vecs = V[batch_indices]  # (batch_size, dim)
        # Compute similarity: batch_vecs @ V.T -> (batch_size, n_total)
        sim = batch_vecs @ V.T
        
        # Exclude self: for each sampled row j, set sim[j_local, batch_indices[j_local]] = -inf
        j_local = torch.arange(batch_size, device=_device)
        self_indices = torch.tensor(batch_indices, device=_device)
        sim[j_local, self_indices] = -1e9
        
        # Get top-500 for each sampled row
        _, topk_idx = sim.topk(min(TOP_K, sim.shape[1]), dim=1)  # (batch_size, TOP_K)
        
        # Build pairs: (similar_row_idx, sampled_row_idx)
        sampled_row_indices = torch.tensor(batch_indices, device=_device).repeat_interleave(TOP_K)
        pairs_batch = torch.stack([topk_idx.flatten(), sampled_row_indices], dim=1)
        pairs_bert.extend(pairs_batch.cpu().tolist())
    else:
        # CPU fallback
        batch_vecs = V[batch_indices]
        sim = batch_vecs @ V.T
        for j_local, j in enumerate(batch_indices):
            col = np.asarray(sim[j_local, :]).ravel()
            col[j] = -np.inf
            kth = min(TOP_K, col.size) - 1
            if kth < 0:
                continue
            top500 = np.argpartition(-col, kth)[:TOP_K]
            for i in top500:
                pairs_bert.append((int(i), j))

print(f"\nAfter sentence-BERT blocking: {len(pairs_bert):,} candidate pairs")
print(f"Expected file size: ~{len(pairs_bert) * 1353 / (1024**3):.1f} GB (based on ~1.3 KB per pair)")

Using device for encoding and top-k: cuda


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 576.00it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Total records: 1,388,738
Encoding all records with sentence-BERT (batched)...


Batches: 100%|██████████| 2713/2713 [01:31<00:00, 29.62it/s]


Freed encoder model; GPU memory available for blocking.

Sampling 100 rows (out of 1,388,738) WITHOUT replacement
Each sampled row will generate top-1000 similar pairs
Expected total: ~100,000 pairs (~100,000 target)
Target: <30 min training time on RTX 5090


Top-1000 blocking: 100%|██████████| 1/1 [00:00<00:00, 73.72it/s]


After sentence-BERT blocking: 100,000 candidate pairs
Expected file size: ~0.1 GB (based on ~1.3 KB per pair)


In [57]:
# Step 4: Label pairs (same canonical_int = 1) and split into train/test/valid
labeled_pairs = []
for i, j in pairs_bert:
    label = 1 if canonical_ints[i] == canonical_ints[j] and canonical_ints[i] != 0 else 0
    labeled_pairs.append((i, j, label))

labeled_pairs = np.array(labeled_pairs, dtype=object)
labels = np.array([x[2] for x in labeled_pairs])
n_pos = int(np.sum(labels == 1))
n_neg = int(np.sum(labels == 0))

print(f"Labeled pairs: {len(labeled_pairs)}")
print(f"Matches (label=1): {n_pos} ({100*n_pos/len(labeled_pairs):.1f}%)")
print(f"Non-matches (label=0): {n_neg} ({100*n_neg/len(labeled_pairs):.1f}%)")

# Stratified split: 80% train, 10% valid, 10% test
SEED_BLOCKING = 42
np.random.seed(SEED_BLOCKING)
pos_idx = np.where(labels == 1)[0]
neg_idx = np.where(labels == 0)[0]
np.random.shuffle(pos_idx)
np.random.shuffle(neg_idx)

def split_indices(idxs, train_ratio=0.8, valid_ratio=0.1):
    n = len(idxs)
    if n == 0:
        return [], [], []
    t = max(1, int(n * train_ratio))
    v = max(0, int(n * valid_ratio))
    te = n - t - v
    return idxs[:t], idxs[t:t+v], idxs[t+v:]

pos_t, pos_v, pos_te = split_indices(pos_idx)
neg_t, neg_v, neg_te = split_indices(neg_idx)

train_idx = np.concatenate([pos_t, neg_t])
valid_idx = np.concatenate([pos_v, neg_v])
test_idx = np.concatenate([pos_te, neg_te])
np.random.shuffle(train_idx)
np.random.shuffle(valid_idx)
np.random.shuffle(test_idx)

print(f"\nSplit sizes:")
print(f"  Train: {len(train_idx)}")
print(f"  Valid: {len(valid_idx)}")
print(f"  Test: {len(test_idx)}")

Labeled pairs: 100000
Matches (label=1): 34222 (34.2%)
Non-matches (label=0): 65778 (65.8%)

Split sizes:
  Train: 79999
  Valid: 9999
  Test: 10002


In [58]:
# Step 5: Write train.txt, test.txt, valid.txt
OUTPUT_DIR_BLOCKING = os.path.join(DITTO_DIR, "data/ijf/blocking_sentence_bert")
os.makedirs(OUTPUT_DIR_BLOCKING, exist_ok=True)

def write_split(split_name, indices):
    path = os.path.join(OUTPUT_DIR_BLOCKING, f"{split_name}.txt")
    with open(path, "w", encoding="utf-8") as f:
        for idx in indices:
            i, j, label = labeled_pairs[idx]
            rec1 = records_blocking[i][1]
            rec2 = records_blocking[j][1]
            f.write(f"{rec1}\t{rec2}\t{label}\n")
    return path

train_path = write_split("train", train_idx)
valid_path = write_split("valid", valid_idx)
test_path = write_split("test", test_idx)

print(f"Wrote train.txt, valid.txt, test.txt to {OUTPUT_DIR_BLOCKING}")

# Report statistics
train_labels = np.array([labeled_pairs[i][2] for i in train_idx])
valid_labels = np.array([labeled_pairs[i][2] for i in valid_idx])
test_labels_blocking = np.array([labeled_pairs[i][2] for i in test_idx])

print(f"\n=== Dataset Statistics ===")
print(f"Train: {len(train_idx)} pairs | Matches: {int(train_labels.sum())} ({100*train_labels.mean():.1f}%) | Non-matches: {len(train_idx)-int(train_labels.sum())} ({100*(1-train_labels.mean()):.1f}%)")
print(f"Valid: {len(valid_idx)} pairs | Matches: {int(valid_labels.sum())} ({100*valid_labels.mean():.1f}%) | Non-matches: {len(valid_idx)-int(valid_labels.sum())} ({100*(1-valid_labels.mean()):.1f}%)")
print(f"Test:  {len(test_idx)} pairs | Matches: {int(test_labels_blocking.sum())} ({100*test_labels_blocking.mean():.1f}%) | Non-matches: {len(test_idx)-int(test_labels_blocking.sum())} ({100*(1-test_labels_blocking.mean()):.1f}%)")

Wrote train.txt, valid.txt, test.txt to /workspace/Ditto/FAIR-DA4ER/FAIR-DA4ER/ditto/data/ijf/blocking_sentence_bert

=== Dataset Statistics ===
Train: 79999 pairs | Matches: 27377 (34.2%) | Non-matches: 52622 (65.8%)
Valid: 9999 pairs | Matches: 3422 (34.2%) | Non-matches: 6577 (65.8%)
Test:  10002 pairs | Matches: 3423 (34.2%) | Non-matches: 6579 (65.8%)


In [64]:

# Step 6: Load datasets and create model
print("Step 6: Loading datasets and creating model...")
train_ds_blocking = DittoDataset(train_path, tokenizer, max_len=MAX_LEN)
valid_ds_blocking = DittoDataset(valid_path, tokenizer, max_len=MAX_LEN)
test_ds_blocking = DittoDataset(test_path, tokenizer, max_len=MAX_LEN)

# Optimized batch sizes for RTX 5090 (much faster training)
BATCH_SIZE_TRAIN = 64   # Increased from 8 → 64 (8x larger batches = 8x fewer iterations)
BATCH_SIZE_EVAL = 128   # Larger for validation/test (faster evaluation)

# DataLoader optimizations: num_workers for parallel loading, pin_memory for faster GPU transfer
train_loader_blocking = DataLoader(
    train_ds_blocking, 
    batch_size=BATCH_SIZE_TRAIN, 
    shuffle=True, 
    collate_fn=DittoDataset.collate_fn,
    num_workers=4,        # Parallel data loading
    pin_memory=True,      # Faster GPU transfer
    persistent_workers=True  # Keep workers alive between epochs
)
valid_loader_blocking = DataLoader(
    valid_ds_blocking, 
    batch_size=BATCH_SIZE_EVAL, 
    shuffle=False, 
    collate_fn=DittoDataset.collate_fn,
    num_workers=2,
    pin_memory=True
)
test_loader_blocking = DataLoader(
    test_ds_blocking, 
    batch_size=BATCH_SIZE_EVAL, 
    shuffle=False, 
    collate_fn=DittoDataset.collate_fn,
    num_workers=2,
    pin_memory=True
)

# Create new model for this run
model_blocking = DittoModel(lm_name=LM_NAME).to(device)
optimizer_blocking = AdamW(model_blocking.parameters(), lr=LR)
num_steps_blocking = len(train_loader_blocking) 
scheduler_blocking = get_linear_schedule_with_warmup(optimizer_blocking, num_warmup_steps=0, num_training_steps=num_steps_blocking)
criterion_blocking = nn.CrossEntropyLoss()

print(f"Train dataset: {len(train_ds_blocking):,} pairs")
print(f"Valid dataset: {len(valid_ds_blocking):,} pairs")
print(f"Test dataset: {len(test_ds_blocking):,} pairs")
print()
print("Optimized Training Settings:")
print(f"  Batch size (train): {BATCH_SIZE_TRAIN} (was 8)")
print(f"  Batch size (eval): {BATCH_SIZE_EVAL} (was 16)")
print(f"  Batches per epoch (train): {len(train_loader_blocking):,} (was {len(train_ds_blocking)//8:,})")
print(f"  DataLoader workers: 4 (parallel loading)")
print(f"  Pin memory: True (faster GPU transfer)")
print()
print("Estimated Training Time (RTX 5090):")
print(f"  Per epoch: ~3-4 minutes")
print(f"  Total ({N_EPOCHS} epochs): ~15-20 minutes")
print(f"  Speedup: ~14-28x faster than batch_size=8")


Step 6: Loading datasets and creating model...


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 498.49it/s, Materializing param=transformer.layer.5.sa_layer_norm.weight]   
DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Train dataset: 79,999 pairs
Valid dataset: 9,999 pairs
Test dataset: 10,002 pairs

Optimized Training Settings:
  Batch size (train): 64 (was 8)
  Batch size (eval): 128 (was 16)
  Batches per epoch (train): 1,250 (was 9,999)
  DataLoader workers: 4 (parallel loading)
  Pin memory: True (faster GPU transfer)

Estimated Training Time (RTX 5090):
  Per epoch: ~3-4 minutes
  Total (5 epochs): ~15-20 minutes
  Speedup: ~14-28x faster than batch_size=8


In [67]:
# Step 7: Training loop
set_seed(SEED_BLOCKING)
best_dev_f1_blocking = 0.0
best_test_f1_blocking = 0.0

for epoch in range(1, N_EPOCHS + 1):
    loss = train_epoch(
        model_blocking,
        train_loader_blocking,
        optimizer_blocking,
        scheduler_blocking,
        criterion_blocking,
    )
    dev_f1, dev_th, _, _ = evaluate(model_blocking, valid_loader_blocking)
    test_f1, _, _, _ = evaluate(model_blocking, test_loader_blocking, threshold=dev_th)

    if dev_f1 > best_dev_f1_blocking:
        best_dev_f1_blocking = dev_f1
        best_test_f1_blocking = test_f1

    print(
        f"Epoch {epoch} | loss={loss:.4f} "
        f"| dev_f1={dev_f1:.4f} | test_f1={test_f1:.4f} "
        f"| best_test_f1={best_test_f1_blocking:.4f}"
    )

Epoch 1 | loss=0.0034 | dev_f1=0.9985 | test_f1=0.9988 | best_test_f1=0.9988
Epoch 2 | loss=0.0035 | dev_f1=0.9985 | test_f1=0.9988 | best_test_f1=0.9988
Epoch 3 | loss=0.0037 | dev_f1=0.9985 | test_f1=0.9988 | best_test_f1=0.9988
Epoch 4 | loss=0.0033 | dev_f1=0.9985 | test_f1=0.9988 | best_test_f1=0.9988
Epoch 5 | loss=0.0038 | dev_f1=0.9985 | test_f1=0.9988 | best_test_f1=0.9988


In [68]:
# Step 8: Final evaluation on test set with detailed metrics
dev_f1_blocking, dev_th_blocking, _, _ = evaluate(model_blocking, valid_loader_blocking)
test_f1_blocking, _, test_preds_blocking, test_labels_blocking_final = evaluate(model_blocking, test_loader_blocking, threshold=dev_th_blocking)

# Compute TP, FP, TN, FN
y_true = np.array(test_labels_blocking_final)
y_pred = np.array(test_preds_blocking)

tp = int(np.sum((y_true == 1) & (y_pred == 1)))
fp = int(np.sum((y_true == 0) & (y_pred == 1)))
tn = int(np.sum((y_true == 0) & (y_pred == 0)))
fn = int(np.sum((y_true == 1) & (y_pred == 0)))

# Compute metrics
accuracy = (tp + tn) / len(y_true) if len(y_true) > 0 else 0.0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

print("=== Final Test Set Performance (IJF Blocking, sentence-BERT top-20) ===")
print(classification_report(y_true, y_pred, target_names=["No match", "Match"]))
print(f"\n=== Detailed Metrics ===")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1:        {f1:.4f}")
print(f"\n=== Confusion Matrix ===")
print(f"TP (True Positives):  {tp}")
print(f"FP (False Positives): {fp}")
print(f"TN (True Negatives):  {tn}")
print(f"FN (False Negatives): {fn}")
print(f"\nTotal test pairs: {len(y_true)}")

=== Final Test Set Performance (IJF Blocking, sentence-BERT top-20) ===
              precision    recall  f1-score   support

    No match       1.00      1.00      1.00      6579
       Match       1.00      1.00      1.00      3423

    accuracy                           1.00     10002
   macro avg       1.00      1.00      1.00     10002
weighted avg       1.00      1.00      1.00     10002


=== Detailed Metrics ===
Accuracy:  0.9992
Precision: 0.9977
Recall:    1.0000
F1:        0.9988

=== Confusion Matrix ===
TP (True Positives):  3423
FP (False Positives): 8
TN (True Negatives):  6571
FN (False Negatives): 0

Total test pairs: 10002


In [69]:
# Step 9: Show examples of TP, FP, TN, FN
tp_idx_blocking = np.where((y_true == 1) & (y_pred == 1))[0]
fp_idx_blocking = np.where((y_true == 0) & (y_pred == 1))[0]
tn_idx_blocking = np.where((y_true == 0) & (y_pred == 0))[0]
fn_idx_blocking = np.where((y_true == 1) & (y_pred == 0))[0]

print(f"\n=== Example Analysis ===")
show_examples(test_ds_blocking, tp_idx_blocking, "True Positives (Blocking)", k=3)
show_examples(test_ds_blocking, fp_idx_blocking, "False Positives (Blocking)", k=3)
show_examples(test_ds_blocking, tn_idx_blocking, "True Negatives (Blocking)", k=3)
show_examples(test_ds_blocking, fn_idx_blocking, "False Negatives (Blocking)", k=3)


=== Example Analysis ===


NameError: name 'show_examples' is not defined

In [70]:
# Inspect FP, FN, TP, TN examples on the IJF sentence-bert subset
import numpy as np

# Convert labels/predictions to numpy arrays
y_true = np.array(test_labels)
y_pred = np.array(test_preds)

# Indices for each case
tp_idx = np.where((y_true == 1) & (y_pred == 1))[0]
fp_idx = np.where((y_true == 0) & (y_pred == 1))[0]
tn_idx = np.where((y_true == 0) & (y_pred == 0))[0]
fn_idx = np.where((y_true == 1) & (y_pred == 0))[0]

print(f"TP: {len(tp_idx)}, FP: {len(fp_idx)}, TN: {len(tn_idx)}, FN: {len(fn_idx)}")


def show_examples(ds, idxs, kind, k=5):
    """Pretty-print up to k examples for a given index set."""
    print(f"\n=== {kind} examples (showing up to {k}) ===")
    for idx in idxs[:k]:
        rec1, rec2 = ds.pairs[idx]
        label = ds.labels[idx]
        print(f"\nIndex: {idx}")
        print(f"True label: {label}")
        print(f"Record 1: {rec1}")
        print(f"Record 2: {rec2}")

# Show a few examples of each type from the test set
show_examples(test_ds, tp_idx, "True Positives")
show_examples(test_ds, fp_idx, "False Positives")
show_examples(test_ds, tn_idx, "True Negatives")
show_examples(test_ds, fn_idx, "False Negatives")

NameError: name 'test_labels' is not defined

In [ ]:
# Training loop (Ditto with DA)
best_dev_f1_da = 0.0
best_test_f1_da = 0.0
for epoch in range(1, N_EPOCHS + 1):
    loss = train_epoch_da(model_da, train_loader_da, optimizer_da, scheduler_da, criterion_da)
    dev_f1, dev_th, _, _ = evaluate_da(model_da, valid_loader_da)
    test_f1, _, _, _ = evaluate_da(model_da, test_loader_da, threshold=dev_th)
    if dev_f1 > best_dev_f1_da:
        best_dev_f1_da = dev_f1
        best_test_f1_da = test_f1
    print(f"Epoch {epoch} | loss={loss:.4f} | dev_f1={dev_f1:.4f} | test_f1={test_f1:.4f} | best_test_f1={best_test_f1_da:.4f}")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1 | loss=0.5796 | dev_f1=0.5806 | test_f1=0.5926 | best_test_f1=0.5926
Epoch 2 | loss=0.4715 | dev_f1=0.6316 | test_f1=0.6667 | best_test_f1=0.6667
Epoch 3 | loss=0.2959 | dev_f1=0.8475 | test_f1=0.8621 | best_test_f1=0.8621
Epoch 4 | loss=0.1591 | dev_f1=0.8966 | test_f1=0.9123 | best_test_f1=0.9123
Epoch 5 | loss=0.1178 | dev_f1=0.8889 | test_f1=0.8889 | best_test_f1=0.9123
Epoch 6 | loss=0.0505 | dev_f1=0.8889 | test_f1=0.9091 | best_test_f1=0.9123
Epoch 7 | loss=0.1673 | dev_f1=0.8814 | test_f1=0.8966 | best_test_f1=0.9123
Epoch 8 | loss=0.0305 | dev_f1=0.8772 | test_f1=0.9310 | best_test_f1=0.9123
Epoch 9 | loss=0.0241 | dev_f1=0.8772 | test_f1=0.9310 | best_test_f1=0.9123
Epoch 10 | loss=0.0134 | dev_f1=0.8772 | test_f1=0.9123 | best_test_f1=0.9123


In [ ]:
# Final test report (Ditto with DA) — same format as before
dev_f1_da, dev_th_da, _, _ = evaluate_da(model_da, valid_loader_da)
test_f1_da, _, test_preds_da, test_labels_da = evaluate_da(model_da, test_loader_da, threshold=dev_th_da)
print(f"Part B — {TASK_NAME_DA} (with data augmentation):")
print("Final test set performance (threshold from validation):")
print(classification_report(test_labels_da, test_preds_da, target_names=["No match", "Match"]))
print(f"F1: {f1_score(test_labels_da, test_preds_da):.4f}, Precision: {precision_score(test_labels_da, test_preds_da):.4f}, Recall: {recall_score(test_labels_da, test_preds_da):.4f}")

NameError: name 'evaluate_da' is not defined